**Basic functions for parity bit method...**

In [1]:
# Functions to convert message to binary using binary conversion of ASCII values
# Converting decimal to binary
def decToBin(n):
    b = ''
    while n > 0:
        b = str(n % 2) + b
        n //= 2
    return b
# Converting string to binary
def stringToBin(s):
    b = ''
    for c in s:
        # Binary representation of character
        k = decToBin(ord(c))
        # Padding the binary representation to have 7 digits
        for i in range(0, 7 - len(k)):
            k = '0' + k
        # Adding to binary message
        b = b +  k
    return b
#====================================
# Functions to check the actual message in character form
# Converting binary to decimal
def binToDec(b):
    n, multiplier = 0, 1
    for i in range(-1, -len(b) - 1, -1):
        n += multiplier*int(b[i])
        multiplier *= 2
    return n
# Converting binary to text string
def binToString(b):
    s = ""
    for i in range(0, len(b) - 6, 7):
        # Obtaining the character for the binary number
        c = chr(binToDec(b[i:i+7]))
        # Adding to string
        s += c
    return s
#====================================
# Function to simulate single bit error
from random import randint
def errorify(s):
    # NOTE: We assume s is the message in binary form!
    #------------------------
    # Choosing a random bit to change
    i = randint(0, len(s) - 1)
    # Creating error
    s = s[:i] + str((int(s[i]) + 1) % 2) + s[i+1:]
    """
    str((int(s[i]) + 1) % 2) converts 1 to 0 and 0 to 1.
    """
    return s

Testing above functions...

In [3]:
print(decToBin(13))
print(stringToBin("Hi"))
print("------------")
print(binToDec("1011011"))
print(binToString(stringToBin("Hi")))
print("------------")
x = errorify(stringToBin("Hi"))
print(x)
print(binToString(x))

1101
10010001101001
------------
91
Hi
------------
10110001101001
Xi


**PARITY BIT METHOD USING HAMMING CODE** (even parity)

**Functions specific for implementing Hamming code...**

In [4]:
# Inserting redundant bits among data bits
def insertRedundantBitPositions(s, printInfo=False):
    """
    printInfo is an optional boolean argument.
    It is only used to enable code output demonstration.
    By default, it is false.
    """
    
    # Converting to binary form
    s = stringToBin(s)
    # Converting string to list (of 1's and 0's)
    s = list(s)
    #------------------------
    # Inserting redundant bits at positions of power 2
    # Inserting parity bits at positions of power 2
    t = []
    cur = 1 # Current redundant bit position
    prev = 0 # Previous redundant bit position
    redundantBitCount, i = 0, 0
    """
    In Hamming code,
    2^r >= m+r+1, where
    m => number of data bits
    r => number of redundant bits
    
    Hence, we need to keep adding
    redundant bits until 2^r >= m+r+1.
    Hence, while 2^r < m+r+1,
    we need to keep adding redundant bits
    """
    while 2**redundantBitCount < (len(s) + redundantBitCount + 1):
        # Appending data bits between two redundant bit positions
        """
        Note that we place data bits until the next
        redundant bit. To do this, the data bit index,
        i, must rise within the bounds of the current
        and previous redundant bits.
        
        Since in this code the redundant are given as
        positions (for calculation reasons) while an array
        index is essentially position-1
        (i.e. position 1 => index 0, etc.),
        we subtract 1 from the difference cur-prev to
        provide a limited increase for the data bit index.
        
        Note that cur-prev-1 provides the upper bound to
        the increase in i from its current value.
        Hence, the actual upper bound for i would be
        i + cur-prev-1.
        """
        upper = i + cur-prev-1
        while i < upper:
            try:
                t.append(s[i])
                i += 1
            except:
                """
                Exception occurs if all data bits
                have been added to the list t.
                """
                return (t, positions)
        
        # Appending redundant bit position
        t.append(" ")
        
        # Changing variables
        redundantBitCount += 1
        prev = cur
        cur *= 2
    #------------------------
    # Appending the remaining data bits
    while i < len(s):
        t.append(s[i])
        i += 1
    #------------------------
    # Printing data for reference
    if printInfo:
        print("Ordered list of data bits among redundant bits:")
        print(t)
        print("Redundant bit count:", redundantBitCount)
    #------------------------
    return (t, redundantBitCount)
#=====================================
# Inserting parity bits in the redundant bit positions
# Inserting number of ones in the respective positions for each redundant bit
def insertTotalOnes(s, printInfo=False):
    """
    printInfo is an optional boolean argument.
    It is only used to enable code output demonstration.
    By default, it is false.
    """
    
    # Obtaining list of data bits and redundant bits
    # as well as redundant bit count
    s, redundantBitCount = insertRedundantBitPositions(s)
    # Getting array of positions (position = index + 1)
    positions = list(range(1, len(s)+1))
    # Printing for demo purposes
    if printInfo:
        print("Positions in binary form (for reference):")
        print(list(map(decToBin, positions)))
    #------------------------
    cur = 1 # Current redundant bit position
    for i in range(0, redundantBitCount):
        # Totalling ones from every position
        # (whose binary form has 1 in the ith place)
        """
        ith place => ith position from the right
        """
        totalOnes, nextRedundantBitPosition = 0, 1
        for j in range(0, len(positions)):
            # Skipping next redundant bit position's corresponding index
            if j == nextRedundantBitPosition-1:
                nextRedundantBitPosition *= 2
                continue
            # Checking if the least significant binary digit is 1
            if positions[j]&1 == 1:
                totalOnes += int(s[j])
            # Shifting position number to the right
            # (i.e. dividing by 2)
            """
            This is to move to the next least
            significant digit in the binary
            representation of the position number.
            """
            positions[j] = positions[j]>>1
        s[cur-1] = totalOnes
        cur *= 2
    #------------------------
    # Printing data and filled redundant bit list for reference
    if printInfo:
        print("Ordered list of data and filled redundant bits:")
        print(s)
    #------------------------
    return (s, redundantBitCount)

# Obtaining the parity bits
def insertParityBits(s, printInfo=False):
    s, redundantBitCount = insertTotalOnes(s)
    cur = 1 # Current redundant/parity bit position
    for i in range(0, redundantBitCount):
        s[cur-1] %= 2
        # Converting to string for demo purposes
        s[cur-1] = str(s[cur-1])
        cur *= 2
    #------------
    # Printing data and parity bit list for reference
    if printInfo:
        print("Ordered list of data and parity bits:")
        print(s)
    return s
#=====================================
# Functions to check the actual message in character form
def getDataBits(s):
    dataBits = []
    nextRedundantBitPosition = 1
    for j in range(0, len(s)):
        # Skipping next redundant bit position's corresponding index
        if j == nextRedundantBitPosition-1:
            nextRedundantBitPosition *= 2
            continue
        # Checking if the least significant binary digit is 1
        dataBits.append(s[j])
    return dataBits
#=====================================
# Function to detect single bit errors using Hamming code parity bit method
def detectError(s):
    positions = list(range(1, len(s)+1))
    cur = 1 # Current parity bit position
    while cur <= len(s):
        # Totalling ones from every position
        # (whose binary form has 1 in the ith place)
        """
        ith place => ith position from the right
        """
        totalOnes = 0
        for j in range(0, len(positions)):
            totalOnes, nextRedundantBitPosition = 0, 1
        for j in range(0, len(positions)):
            # Skipping next redundant bit position's corresponding index
            if j == nextRedundantBitPosition-1:
                nextRedundantBitPosition *= 2
                continue
            # Checking if the least significant binary digit is 1
            if positions[j]&1 == 1:
                totalOnes += int(s[j])
            # Shifting position number to the right
            # (i.e. dividing by 2)
            """
            This is to move to the next least
            significant digit in the binary
            representation of the position number.
            """
            positions[j] = positions[j]>>1
        
        if (totalOnes % 2) != int(s[cur-1]):
            print("Single bit error detected for parity bit", cur)
            return True
        cur *= 2
    print("No single bit error detected.")
    return False

Testing to above functions...

In [5]:
print(stringToBin("H"))
x = insertRedundantBitPositions("H", True)
print("------------")
x = insertTotalOnes("H", True)
print("------------")
x = insertParityBits("H", True)
print("------------")
print("Getting back the data bits:")
getDataBits(x)

1001000
Ordered list of data bits among redundant bits:
[' ', ' ', '1', ' ', '0', '0', '1', ' ', '0', '0', '0']
Redundant bit count: 4
------------
Positions in binary form (for reference):
['1', '10', '11', '100', '101', '110', '111', '1000', '1001', '1010', '1011']
Ordered list of data and filled redundant bits:
[2, 2, '1', 1, '0', '0', '1', 0, '0', '0', '0']
------------
Ordered list of data and parity bits:
['0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0']
------------
Getting back the data bits:


['1', '0', '0', '1', '0', '0', '0']

**Application...**

Message sent with parity bits using Hamming code...

In [6]:
txt = "Hola, senor"
msg = insertParityBits(txt)
print("Intended message in plaintext form")
print(binToString(''.join(getDataBits(msg))))
print("Intended message in binary form:")
print(''.join(msg))

Intended message in plaintext form
Hola, senor
Intended message in binary form:
001100110001101111111011001100010101011000100000111001111001011010111011011111110010


Simulating single bit error...

In [10]:
corrupted = list(errorify(''.join(msg)))
print("Corrupted message in plaintext form")
print(binToString(''.join(getDataBits(corrupted))))
print("Corrupted message in binary form:")
print(''.join(corrupted))

Corrupted message in plaintext form
Iola, senor
Corrupted message in binary form:
001100110011101111111011001100010101011000100000111001111001011010111011011111110010


_Note that it may happen that no data bits were corrupted, but instead, one parity bit was changed. In this case, however, it was a data bit that was corrupted._

Message received...

In [11]:
detectError(corrupted)

Single bit error detected for parity bit 1


True

To verify if the detection works, we will test it with the uncorrupted message...

In [12]:
detectError(msg)

No single bit error detected.


False